In [ ]:
!wget https://files.grouplens.org/datasets/movielens/ml-25m.zip
!unzip ml-25m.zip

--2024-04-10 01:31:23--  https://files.grouplens.org/datasets/movielens/ml-25m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261978986 (250M) [application/zip]
Saving to: ‘ml-25m.zip’

ml-25m.zip          100%[===================>] 249.84M  73.9MB/s    in 3.6s    

2024-04-10 01:31:27 (68.6 MB/s) - ‘ml-25m.zip’ saved [261978986/261978986]

Archive:  ml-25m.zip
   creating: ml-25m/
  inflating: ml-25m/tags.csv         
  inflating: ml-25m/links.csv        
  inflating: ml-25m/README.txt       
  inflating: ml-25m/ratings.csv      
  inflating: ml-25m/genome-tags.csv  
  inflating: ml-25m/genome-scores.csv  
  inflating: ml-25m/movies.csv       


# 장르가 유사한 영화 찾기

In [ ]:
import csv

titles = {}
genresets = {}

with open("ml-25m/movies.csv") as f:
    print(f.readline())

    csvreader = csv.reader(f)
    next(csvreader) # skip column names

    for mid, title, genre in csvreader:
        titles[int(mid)] = title
        genresets[int(mid)] = set(genre.split("|"))

movieId,title,genres



In [ ]:
def jaccard_similarity(a, b):
    if len(a|b) == 0: return 0
    return len(a&b) / len(a|b)

def find_topk_jaccard_genres(target_mid, k=20):
    target_title = titles[target_mid]
    target_genres = genresets[target_mid]
    res = []
    for mid, title in titles.items():
        genres = genresets[mid]
        jaccard_score = jaccard_similarity(target_genres, genres)
        # 정렬할 때 편하라고 score를 먼저 넣음.
        res.append( (jaccard_score, title) )

    res.sort(reverse=True)
    return res[:k]


In [ ]:
mid = 164909
res = find_topk_jaccard_genres(mid, 10)
res

[(1.0, 'Собака Павлова (2005)'),
 (1.0, 'Şabaniye (1984)'),
 (1.0, 'Zus & Zo (2001)'),
 (1.0, 'Zero (2018)'),
 (1.0, 'Zack and Miri Make a Porno (2008)'),
 (1.0, 'Youth in Revolt (2009)'),
 (1.0, 'You and the Night (2013)'),
 (1.0, 'You Stupid Man (2002)'),
 (1.0, 'You Are the Apple of My Eye (2011)'),
 (1.0, 'Yo Yo (Yoyo) (1965)')]

# 다른 사용자가 함께 본 영화 찾기

In [ ]:
ratings = []

with open("ml-25m/ratings.csv", "r") as f:
    print(f.readline()) # skip column names

    for line in f:
        uid, mid, rating, timestamp = line.split(",")
        ratings.append((int(uid), int(mid), float(rating)))

userId,movieId,rating,timestamp



In [ ]:
# jaccard similarity with rating history
from collections import defaultdict
# 딕셔너리로 만듦.
usets = defaultdict(set)

for uid, mid, rating in ratings:
    usets[mid].add(uid)

In [ ]:
first_item = next(iter(usets.items()))
print(first_item)


(296, {1, 3, 4, 5, 7, 8, 10, 12, 13, 14, 15, 18, 19, 22, 23, 25, 26, 31, 32, 33, 34, 37, 38, 40, 43, 45, 50, 56, 57, 59, 62, 68, 69, 70, 72, 74, 75, 76, 79, 80, 82, 84, 85, 86, 94, 95, 99, 101, 103, 104, 106, 107, 109, 111, 112, 113, 114, 118, 120, 122, 124, 125, 126, 127, 132, 133, 135, 136, 137, 139, 142, 147, 148, 152, 155, 156, 158, 160, 165, 166, 167, 168, 171, 174, 175, 178, 179, 181, 185, 186, 187, 190, 196, 197, 199, 201, 202, 204, 205, 206, 207, 208, 211, 214, 216, 217, 220, 224, 226, 227, 228, 229, 230, 231, 232, 233, 235, 236, 238, 239, 240, 243, 245, 246, 247, 249, 250, 255, 258, 259, 262, 264, 265, 267, 269, 272, 273, 279, 280, 281, 284, 285, 290, 293, 294, 296, 301, 302, 304, 306, 309, 310, 311, 312, 315, 318, 319, 320, 321, 323, 328, 329, 331, 334, 337, 342, 343, 344, 345, 350, 352, 355, 356, 357, 358, 362, 363, 364, 366, 368, 371, 372, 374, 375, 376, 378, 379, 382, 386, 388, 390, 395, 396, 397, 398, 399, 401, 404, 408, 409, 411, 413, 414, 416, 417, 419, 420, 426, 427, 4

In [ ]:
from tqdm import tqdm
def find_topk_jaccard_ratings(target_mid, k=20):
    target_uset = usets[target_mid]

    res = []

    for mid, title in tqdm(titles.items()):
        uset = usets[mid]
        jaccard_score = jaccard_similarity(target_uset, uset)
        res.append( (jaccard_score, title) )

    res.sort(reverse=True)
    return res[:k]

In [ ]:
mid = 112552
res = find_topk_jaccard_ratings(mid)
res

[(1.0, 'Whiplash (2014)'),
 (0.34346657709103123, 'Gone Girl (2014)'),
 (0.33069178628389156, 'The Imitation Game (2014)'),
 (0.3202530162820663, 'Interstellar (2014)'),
 (0.31890598662389213, 'Grand Budapest Hotel, The (2014)'),
 (0.3178914625340221, 'Ex Machina (2015)'),
 (0.31770758896670004,
  'Birdman: Or (The Unexpected Virtue of Ignorance) (2014)'),
 (0.3133357646486749, 'Her (2013)'),
 (0.312987900078064, 'Wolf of Wall Street, The (2013)'),
 (0.3059296340689558, 'Nightcrawler (2014)'),
 (0.2975845919065411, 'Mad Max: Fury Road (2015)'),
 (0.2957345586142398, 'Django Unchained (2012)'),
 (0.2737903411445145, 'The Martian (2015)'),
 (0.2668467981626587, 'Gravity (2013)'),
 (0.2627047459050819, 'Shutter Island (2010)'),
 (0.2575789399985997, 'Dallas Buyers Club (2013)'),
 (0.25711628659226915, 'Dark Knight Rises, The (2012)'),
 (0.25615628009354796, 'The Revenant (2015)'),
 (0.2561167699254437, 'Intouchables (2011)'),
 (0.25411334552102377, 'Arrival (2016)')]

In [ ]:
ursets = defaultdict(dict)

for uid, mid, rating in ratings:
    ursets[mid][uid] = rating

In [ ]:
for mid, urset in ursets.items():
    avg = sum(urset.values())/ len(urset)
    for k in urset:
        urset[k] -= avg

In [ ]:
def cosine_similarity(a, b):
    numerator = sum(a[k] * b[k] for k in a.keys() & b.keys())
    de = (sum(x*x for x in a.values()) * sum(x*x for x in b.values())) **0.5
    if de == 0: return 0
    return numerator / de

In [ ]:
from tqdm import tqdm
def find_topk_pearson_ratings(target_mid, k=20):
    target_uset = ursets[target_mid]

    res = []

    for mid, title in tqdm(titles.items()):
        uset = ursets[mid]
        cosine_score = cosine_similarity(target_uset, urset)
        res.append( (cosine_score, title) )

    res.sort(reverse=True)
    return res[:k]